In [ ]:
# Programa para descargar CVs de candidatos y organizarlos por Poder de la Unión
# Para usar en Google Colab

import pandas as pd
import requests
import os
import io
from google.colab import drive
import re
from urllib.parse import urlparse
from tqdm.notebook import tqdm
import mimetypes

# Montar Google Drive
drive.mount('/content/drive')

# Definir la ruta base donde se guardarán los CVs
BASE_PATH = '/content/drive/My Drive/Colab Notebooks/AnalisisCandidatosPoderJudicialdelaFederación/CVs'

# Crear la carpeta base si no existe
os.makedirs(BASE_PATH, exist_ok=True)

# Leer el archivo Excel
def leer_excel(ruta_archivo):
    try:
        return pd.read_excel(ruta_archivo)
    except Exception as e:
        print(f"Error al leer el archivo Excel: {e}")
        return None

# Determinar el tipo de archivo basado en la URL o encabezados de respuesta
def obtener_extension_archivo(url, response=None):
    # Primero intentar obtener la extensión desde la URL
    parsed_url = urlparse(url)
    path = parsed_url.path
    extension = os.path.splitext(path)[1].lower()

    if extension and extension != '.':
        return extension

    # Si no hay extensión en la URL o es inválida, intentar con el Content-Type
    if response and 'Content-Type' in response.headers:
        content_type = response.headers['Content-Type'].split(';')[0].strip()
        ext = mimetypes.guess_extension(content_type)
        if ext:
            return ext

    # Si todo falla, asumimos que es un PDF (formato común para CVs)
    return '.pdf'

# Función para limpiar nombres de archivo
def limpiar_nombre_archivo(nombre):
    # Eliminar caracteres no permitidos en nombres de archivo
    nombre_limpio = re.sub(r'[\\/*?:"<>|]', "", nombre)
    # Reemplazar espacios por guiones bajos
    nombre_limpio = nombre_limpio.replace(' ', '_')
    # Limitar longitud del nombre
    if len(nombre_limpio) > 120:
        nombre_limpio = nombre_limpio[:120]
    return nombre_limpio

# Descargar y guardar CV
def descargar_cv(url, candidato, poder, edo, cargo):
    try:
        if not url or pd.isna(url):
            print(f"URL no válida para el candidato {candidato}")
            return False

        print(f"Descargando CV de: {candidato} - {url}")

        # Hacer la solicitud HTTP
        response = requests.get(url, stream=True, timeout=30)

        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            # Obtener la extensión del archivo
            extension = obtener_extension_archivo(url, response)

            # Crear el nombre del archivo
            nombre_archivo = f"{limpiar_nombre_archivo(candidato)}{extension}"

            # Crear la carpeta para este poder si no existe
            poder_directorio = os.path.join(BASE_PATH, limpiar_nombre_archivo(poder))
            os.makedirs(poder_directorio, exist_ok=True)

            # Ruta completa donde se guardará el archivo
            ruta_completa = os.path.join(poder_directorio, nombre_archivo)

            # Guardar el archivo
            with open(ruta_completa, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

            print(f"CV guardado en: {ruta_completa}")

            # Crear archivo de metadatos (información adicional sobre el candidato)
            ruta_metadata = os.path.join(poder_directorio, f"{limpiar_nombre_archivo(candidato)}_metadata.txt")
            with open(ruta_metadata, 'w', encoding='utf-8') as f:
                f.write(f"Candidato: {candidato}\n")
                f.write(f"Poder de la Unión: {poder}\n")
                f.write(f"Estado: {edo}\n")
                f.write(f"Cargo: {cargo}\n")
                f.write(f"URL original: {url}\n")

            return True
        else:
            print(f"Error al descargar CV de {candidato}. Código de estado: {response.status_code}")
            return False

    except Exception as e:
        print(f"Error al procesar CV de {candidato}: {e}")
        return False

# Función principal
def main():
    # Definir la ruta del archivo Excel (ajustar según sea necesario)
    ruta_excel = '/content/drive/My Drive/Colab Notebooks/AnalisisCandidatosPoderJudicialdelaFederación/Candidatos_18-05-2025.xlsx'

    # Preguntar al usuario si desea usar otra ruta
    otra_ruta = input(f"El archivo Excel se buscará en: {ruta_excel}\n¿Desea especificar otra ubicación? (s/n): ")

    if otra_ruta.lower() == 's':
        ruta_excel = input("Ingrese la ruta completa del archivo Excel: ")

    # Leer el archivo Excel
    df = leer_excel(ruta_excel)

    if df is None:
        print("No se pudo leer el archivo Excel. El programa terminará.")
        return

    # Verificar que el DataFrame tenga las columnas necesarias
    columnas_requeridas = ['Edo', 'Cargo', 'Poder_de_la_Unión', 'Candidato', 'Url_del_Curriculum_Candidato']
    columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]

    if columnas_faltantes:
        print(f"El archivo Excel no contiene las siguientes columnas requeridas: {columnas_faltantes}")
        print("Las columnas disponibles son:", df.columns.tolist())
        return

    # Contar cuántos CVs hay por cada Poder de la Unión
    conteo_por_poder = df['Poder_de_la_Unión'].value_counts()
    print("\nDistribución de candidatos por Poder de la Unión:")
    for poder, conteo in conteo_por_poder.items():
        print(f"- {poder}: {conteo} candidatos")

    # Preguntar si desea continuar
    confirmar = input("\n¿Desea continuar con la descarga de los CVs? (s/n): ")
    if confirmar.lower() != 's':
        print("Operación cancelada por el usuario.")
        return

    # Contador de CVs descargados
    cvs_descargados = 0
    total_cvs = len(df)

    # Descargar los CVs con barra de progreso
    print(f"\nIniciando descarga de {total_cvs} CVs...")

    for _, fila in tqdm(df.iterrows(), total=total_cvs, desc="Descargando CVs"):
        resultado = descargar_cv(
            url=fila['Url_del_Curriculum_Candidato'],
            candidato=fila['Candidato'],
            poder=fila['Poder_de_la_Unión'],
            edo=fila['Edo'],
            cargo=fila['Cargo']
        )

        if resultado:
            cvs_descargados += 1

    # Mostrar resumen final
    print(f"\nResumen de la operación:")
    print(f"- Total de candidatos procesados: {total_cvs}")
    print(f"- CVs descargados exitosamente: {cvs_descargados}")
    print(f"- CVs no descargados: {total_cvs - cvs_descargados}")

    if cvs_descargados > 0:
        print(f"\nLos CVs se han guardado en: {BASE_PATH}")
        print("Organizados en carpetas según el Poder de la Unión.")

# Ejecutar el programa
if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
El archivo Excel se buscará en: /content/drive/My Drive/Colab Notebooks/AnalisisCandidatosPoderJudicialdelaFederación/Candidatos_18-05-2025.xlsx
¿Desea especificar otra ubicación? (s/n): n

Distribución de candidatos por Poder de la Unión:
- Poder Legislativo: 47 candidatos
- Poder Ejecutivo: 45 candidatos
- Poder Judicial: 29 candidatos
- En Funciones: 10 candidatos
- Poder Ejecutivo, Poder Legislativo: 7 candidatos
- Poder Ejecutivo, Poder Judicial: 5 candidatos
- Poder Judicial, Poder Legislativo: 5 candidatos
- Poder Ejecutivo, Poder Judicial, Poder Legislativo: 4 candidatos
- En Funciones, Poder Ejecutivo: 1 candidatos

¿Desea continuar con la descarga de los CVs? (s/n): s

Iniciando descarga de 153 CVs...


Descargando CVs:   0%|          | 0/153 [00:00<?, ?it/s]

Descargando CV de: CARRANZA BUENRROSTRO DIANA LAURA - https://candidaturaspoderjudicial.ine.mx/cycc/documentos/ficha/CARRANZA_BUENRROSTRO_DIANA_LAURA_52604.pdf
CV guardado en: /content/drive/My Drive/Colab Notebooks/AnalisisCandidatosPoderJudicialdelaFederación/CVs/Poder_Judicial/CARRANZA_BUENRROSTRO_DIANA_LAURA.pdf
Descargando CV de: MACIAS ALVAREZ ALMA YESENIA - https://candidaturaspoderjudicial.ine.mx/cycc/documentos/ficha/MACIAS_ALVAREZ_ALMA_YESENIA_53721.pdf
CV guardado en: /content/drive/My Drive/Colab Notebooks/AnalisisCandidatosPoderJudicialdelaFederación/CVs/Poder_Legislativo/MACIAS_ALVAREZ_ALMA_YESENIA.pdf
Descargando CV de: OROZCO PRECIADO VERA ISABEL - https://candidaturaspoderjudicial.ine.mx/cycc/documentos/ficha/OROZCO_PRECIADO_VERA_ISABEL_54808.pdf
CV guardado en: /content/drive/My Drive/Colab Notebooks/AnalisisCandidatosPoderJudicialdelaFederación/CVs/Poder_Ejecutivo/OROZCO_PRECIADO_VERA_ISABEL.pdf
Descargando CV de: RODRIGUEZ MENDEZ JOSEFINA - https://candidaturaspoder